In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier


In [39]:
df = pd.read_csv('train.csv')

In [40]:
X = df.iloc[:, :-1]
y = df['Quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [41]:
model_list = [RandomForestClassifier(), LogisticRegression(), SVC(), MLPClassifier(),
             AdaBoostClassifier(), ExtraTreesClassifier(), GradientBoostingClassifier(), HistGradientBoostingClassifier()]

model_name_list = []
model_accuracies = []

for i in model_list:
    model = i.fit(X_train, y_train)
    model_name = model.__class__.__name__
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    model_name_list.append(model_name)
    model_accuracies.append(accuracy)
    
    print(f"{model_name} accuracy: {accuracy:.3f}\n")

RandomForestClassifier accuracy: 0.955

LogisticRegression accuracy: 0.855



/Users/dunhul/Library/CloudStorage/OneDrive-NorwegianUniversityofLifeSciences/VÅR 2025/DAT200/CA3_DAT200/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVC accuracy: 0.507

MLPClassifier accuracy: 0.514

AdaBoostClassifier accuracy: 0.857

ExtraTreesClassifier accuracy: 0.963

GradientBoostingClassifier accuracy: 0.943

HistGradientBoostingClassifier accuracy: 0.961



In [43]:
# Initialize ExtraTreesClassifier
etc = ExtraTreesClassifier(random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees
    'max_depth': [None, 10, 20],  # Tree depth
    'min_samples_split': [2, 5, 10],  # Min samples to split
    'min_samples_leaf': [1, 2, 4],  # Min samples per leaf
    'max_features': ['sqrt', 'log2', None],  # Features for split
    'criterion': ['gini', 'entropy'],  # Splitting criterion
}

# Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(etc, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train model with best parameters
best_etc = ExtraTreesClassifier(**best_params, random_state=42)
best_etc.fit(X_train, y_train)

# Evaluate on test set
y_pred = best_etc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy with Best Hyperparameters:")

Fitting 5 folds for each of 486 candidates, totalling 2430 fits
Best Hyperparameters: {'criterion': 'gini', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


ValueError: Found input variables with inconsistent numbers of samples: [1200, 560]

In [44]:
df_test = pd.read_csv('test.csv')
y_test = best_etc.predict(df_test)
y_test = pd.DataFrame(y_test, columns=["Quality"])
y_test.index.name = "ID"
y_test[['Quality']].to_csv("submission3.csv")